In [1]:
import numpy as np
import pandas as pd
from google.cloud import bigquery
import geopandas as gpd
import h3
import geojson
from geojson import Feature, Point, FeatureCollection, Polygon
import plotly.express as px
from shapely import Polygon
from shapely import wkt
import json

In [2]:
bigquery_client = bigquery.Client(project='geograph-342112')
q = """
    select * from `geograph-342112.kristov_dev.regions_h3_cat_Randall_with_metrics`
"""
query_result = bigquery_client.query(q)
df_regions_to_h3_raw = query_result.to_dataframe()

In [18]:
df_regions_to_h3 = df_regions_to_h3_raw
df_regions_to_h3.head(2)

,location,region_geometry,region_exterior_ring,region_centroid,h3_l7,h3_l5,population,cnt_distinct_h3_l7_in_l5,sum_population_h3_l5,category_Randall,cnt_poi_h3_l7,cnt_poi_h3_l7_in_h3_l5,poi_per_capita_h3_l7,poi_per_capita_h3_l5,poi_per_capita_relative_h3_l7_to_h3_l5
0,London,"POLYGON((-0.5580103 51.7298297, -0.7186854 51....","LINESTRING(-0.5580103 51.7298297, -0.7186854 5...",POINT(-0.138123649968448 51.4791453212843),87194a500ffffff,85194a53fffffff,245.821049,32,21890.376562,Limited-Service Restaurants,0,0,0.0,0.0,0.0
1,London,"POLYGON((-0.5580103 51.7298297, -0.7186854 51....","LINESTRING(-0.5580103 51.7298297, -0.7186854 5...",POINT(-0.138123649968448 51.4791453212843),87194a500ffffff,85194a53fffffff,245.821049,32,21890.376562,Specialty Food Stores,0,0,0.0,0.0,0.0


In [19]:
df_regions_to_h3.groupby(by='category_Randall')['cnt_poi_h3_l7'].sum()

category_Randall
All Other General Merchandise Stores         988
Beer, Wine, and Liquor Stores                143
Building Material and Supplies Dealers      1811
Clothing Stores                             4248
Convenience Stores                          2215
Department Stores                            900
Fitness and Recreational Sports Centers     1609
Full-Service Restaurants                    4098
Gas Stations                                3144
Grocery Stores                              5971
Health and Personal Care Stores             5322
Limited-Service Restaurants                 8721
Others                                     40529
Restaurants and Other Eating Places         5895
Shoe Stores                                 1319
Specialty Food Stores                        403
Sporting Goods, Hobby...                    1233
Universities                                   5
Name: cnt_poi_h3_l7, dtype: Int64

In [20]:
category_to_select = 'Clothing Stores'
df_regions_to_h3 = df_regions_to_h3[df_regions_to_h3['category_Randall'] == category_to_select]

In [21]:
# create id unique id field
df_regions_to_h3['region_h3'] = df_regions_to_h3['location'] + '_' + df_regions_to_h3['h3_l7']
# create indicator variable (to visualize empty hexagons)
df_regions_to_h3['is_null_population'] = np.where(df_regions_to_h3['population'].isna(),1,0)
df_regions_to_h3['is_null_population'] = df_regions_to_h3['is_null_population'].astype('category')
df_regions_to_h3.head(2)

/Users/Andrew/Documents/python/env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/Andrew/Documents/python/env/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/Andrew/Documents/python/env/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

,location,region_geometry,region_exterior_ring,region_centroid,h3_l7,h3_l5,population,cnt_distinct_h3_l7_in_l5,sum_population_h3_l5,category_Randall,cnt_poi_h3_l7,cnt_poi_h3_l7_in_h3_l5,poi_per_capita_h3_l7,poi_per_capita_h3_l5,poi_per_capita_relative_h3_l7_to_h3_l5,region_h3,is_null_population
5,London,"POLYGON((-0.5580103 51.7298297, -0.7186854 51....","LINESTRING(-0.5580103 51.7298297, -0.7186854 5...",POINT(-0.138123649968448 51.4791453212843),87194a500ffffff,85194a53fffffff,245.821049,32,21890.376562,Clothing Stores,0,0,0.0,0.0,0.0,London_87194a500ffffff,0
14,London,"POLYGON((-0.5580103 51.7298297, -0.7186854 51....","LINESTRING(-0.5580103 51.7298297, -0.7186854 5...",POINT(-0.138123649968448 51.4791453212843),87194a501ffffff,85194a53fffffff,273.842666,32,21890.376562,Clothing Stores,0,0,0.0,0.0,0.0,London_87194a501ffffff,0


In [22]:
def normalize_metric_in_location(df, metric):
    df_max = df.groupby(by='location')[metric].max()
    df_merge = df.merge(df_max, how='inner',left_on='location',right_index=True)
    df_merge.rename(columns={f'{metric}_y':f'{metric}_max',f'{metric}_x':f'{metric}'},inplace=True)
    df_merge[f'{metric}_normalized'] = df_merge[f'{metric}'] / df_merge[f'{metric}_max']
    result = df_merge
    return result

df_regions_to_h3 = normalize_metric_in_location(df_regions_to_h3,'cnt_poi_h3_l7')
df_regions_to_h3 = normalize_metric_in_location(df_regions_to_h3,'poi_per_capita_h3_l7')
df_regions_to_h3 = normalize_metric_in_location(df_regions_to_h3,'poi_per_capita_relative_h3_l7_to_h3_l5')
#df_regions_to_h3.head()

In [23]:
# replace na with -1 for visualization
df_regions_to_h3.fillna({'poi_per_capita_relative_h3_l7_to_h3_l5_normalized':-1},inplace=True)
df_regions_to_h3[df_regions_to_h3['population'] == 0].head()

,location,region_geometry,region_exterior_ring,region_centroid,h3_l7,h3_l5,population,cnt_distinct_h3_l7_in_l5,sum_population_h3_l5,category_Randall,...,poi_per_capita_h3_l5,poi_per_capita_relative_h3_l7_to_h3_l5,region_h3,is_null_population,cnt_poi_h3_l7_max,cnt_poi_h3_l7_normalized,poi_per_capita_h3_l7_max,poi_per_capita_h3_l7_normalized,poi_per_capita_relative_h3_l7_to_h3_l5_max,poi_per_capita_relative_h3_l7_to_h3_l5_normalized
3839,Soligorsk,"POLYGON((27.4271169 52.8686898, 27.2939077 52....","LINESTRING(27.4271169 52.8686898, 27.2939077 5...",POINT(27.5265446696382 52.7602771284113),871f4a400ffffff,851f4a43fffffff,0.0,39,0.0,Clothing Stores,...,0.0,0.0,Soligorsk_871f4a400ffffff,0,0,NaN,0.0,NaN,0.0,-1.0
3853,Soligorsk,"POLYGON((27.4271169 52.8686898, 27.2939077 52....","LINESTRING(27.4271169 52.8686898, 27.2939077 5...",POINT(27.5265446696382 52.7602771284113),871f4a401ffffff,851f4a43fffffff,0.0,39,0.0,Clothing Stores,...,0.0,0.0,Soligorsk_871f4a401ffffff,0,0,NaN,0.0,NaN,0.0,-1.0
3884,Soligorsk,"POLYGON((27.4271169 52.8686898, 27.2939077 52....","LINESTRING(27.4271169 52.8686898, 27.2939077 5...",POINT(27.5265446696382 52.7602771284113),871f4a402ffffff,851f4a43fffffff,0.0,39,0.0,Clothing Stores,...,0.0,0.0,Soligorsk_871f4a402ffffff,0,0,NaN,0.0,NaN,0.0,-1.0
3902,Soligorsk,"POLYGON((27.4271169 52.8686898, 27.2939077 52....","LINESTRING(27.4271169 52.8686898, 27.2939077 5...",POINT(27.5265446696382 52.7602771284113),871f4a403ffffff,851f4a43fffffff,0.0,39,0.0,Clothing Stores,...,0.0,0.0,Soligorsk_871f4a403ffffff,0,0,NaN,0.0,NaN,0.0,-1.0
3906,Soligorsk,"POLYGON((27.4271169 52.8686898, 27.2939077 52....","LINESTRING(27.4271169 52.8686898, 27.2939077 5...",POINT(27.5265446696382 52.7602771284113),871f4a404ffffff,851f4a43fffffff,0.0,39,0.0,Clothing Stores,...,0.0,0.0,Soligorsk_871f4a404ffffff,0,0,NaN,0.0,NaN,0.0,-1.0


In [24]:
#df_region_polygons = pd.DataFrame({'geometry':df_regions_to_h3[['location','geometry']].unique()})
df_region_polygons = df_regions_to_h3[['location','region_geometry','region_exterior_ring']].drop_duplicates(keep='first')
df_region_polygons['exterior_ring_shapely'] = wkt.loads(df_region_polygons['region_exterior_ring'])
df_region_polygons

,location,region_geometry,region_exterior_ring,exterior_ring_shapely
5,London,"POLYGON((-0.5580103 51.7298297, -0.7186854 51....","LINESTRING(-0.5580103 51.7298297, -0.7186854 5...","LINESTRING (-0.5580103 51.7298297, -0.7186854 ..."
1947,Birmingham,"POLYGON((-2.2007212 52.6693307, -2.3215708 52....","LINESTRING(-2.2007212 52.6693307, -2.3215708 5...","LINESTRING (-2.2007212 52.6693307, -2.3215708 ..."
3839,Soligorsk,"POLYGON((27.4271169 52.8686898, 27.2939077 52....","LINESTRING(27.4271169 52.8686898, 27.2939077 5...","LINESTRING (27.4271169 52.8686898, 27.2939077 ..."
6292,Minsk,"POLYGON((27.3886889 54.0744289, 27.0920581 53....","LINESTRING(27.3886889 54.0744289, 27.0920581 5...","LINESTRING (27.3886889 54.0744289, 27.0920581 ..."
11300,San Francisco,"POLYGON((-122.5845595 37.7601185, -122.5763197...","LINESTRING(-122.5845595 37.7601185, -122.57631...","LINESTRING (-122.5845595 37.7601185, -122.5763..."
13346,Los Angeles,"POLYGON((-118.662229 34.0412527, -118.5276465 ...","LINESTRING(-118.662229 34.0412527, -118.527646...","LINESTRING (-118.662229 34.0412527, -118.52764..."
18609,Barcelona,"POLYGON((1.9590174 41.2648057, 2.028368 41.258...","LINESTRING(1.9590174 41.2648057, 2.028368 41.2...","LINESTRING (1.9590174 41.2648057, 2.028368 41...."


In [25]:
def h3_to_polygon(h3_hexagon):
    return Polygon(h3.h3_to_geo_boundary(h3_hexagon,geo_json=True))

df_regions_to_h3['h3_polygons'] = df_regions_to_h3['h3_l7'].apply(lambda x: h3_to_polygon(x))
df_regions_to_h3.head(2)

,location,region_geometry,region_exterior_ring,region_centroid,h3_l7,h3_l5,population,cnt_distinct_h3_l7_in_l5,sum_population_h3_l5,category_Randall,...,poi_per_capita_relative_h3_l7_to_h3_l5,region_h3,is_null_population,cnt_poi_h3_l7_max,cnt_poi_h3_l7_normalized,poi_per_capita_h3_l7_max,poi_per_capita_h3_l7_normalized,poi_per_capita_relative_h3_l7_to_h3_l5_max,poi_per_capita_relative_h3_l7_to_h3_l5_normalized,h3_polygons
5,London,"POLYGON((-0.5580103 51.7298297, -0.7186854 51....","LINESTRING(-0.5580103 51.7298297, -0.7186854 5...",POINT(-0.138123649968448 51.4791453212843),87194a500ffffff,85194a53fffffff,245.821049,32,21890.376562,Clothing Stores,...,0.0,London_87194a500ffffff,0,246,0.0,0.017982,0.0,113.71609,0.0,POLYGON ((-0.4384792869340857 51.1916253010286...
14,London,"POLYGON((-0.5580103 51.7298297, -0.7186854 51....","LINESTRING(-0.5580103 51.7298297, -0.7186854 5...",POINT(-0.138123649968448 51.4791453212843),87194a501ffffff,85194a53fffffff,273.842666,32,21890.376562,Clothing Stores,...,0.0,London_87194a501ffffff,0,246,0.0,0.017982,0.0,113.71609,0.0,POLYGON ((-0.4177386497770632 51.1740365939916...


In [26]:
def polygon_dataframe_to_geojson(df,geometry_id,geometry,geometry_value,file_output=None):
    
    """ Function creates geojson for visualization with plotly express"""
    
    list_features = []
    for i,row in df.iterrows():
        feature = Feature(
            geometry = row[geometry] if geometry == 'h3_polygons' else wkt.loads(row[geometry]),
            #geometry = wkt.loads(row[geometry]),
            id = row[geometry_id],
            properties = {'value':row[geometry_value]})
        list_features.append(feature)
    
    feature_collection = FeatureCollection(list_features)
    
    if file_output is not None:
            with open(file_output,'w') as f:
                json.dump(feature_collection,f)
    
    return feature_collection

In [27]:
geojson_object_hexagons = polygon_dataframe_to_geojson(
                            df=df_regions_to_h3,
                            geometry_id='h3_l7',
                            geometry='h3_polygons',
                            geometry_value='is_null_population',
                            file_output='region_hex.json')
geojson_object_polygons = polygon_dataframe_to_geojson(
                            df=df_region_polygons,
                            geometry_id='location',
                            geometry='region_geometry',
                            geometry_value='location',
                            file_output='region_polygons.json')
geojson_object_regions_exterior_lines = polygon_dataframe_to_geojson(
                            df=df_region_polygons,
                            geometry_id='location',
                            geometry='region_exterior_ring',
                            geometry_value='location',
                            file_output='region_exterior_lines.json')

In [32]:
df_regions_to_h3.columns

Index(['location', 'region_geometry', 'region_exterior_ring',
       'region_centroid', 'h3_l7', 'h3_l5', 'population',
       'cnt_distinct_h3_l7_in_l5', 'sum_population_h3_l5', 'category_Randall',
       'cnt_poi_h3_l7', 'cnt_poi_h3_l7_in_h3_l5', 'poi_per_capita_h3_l7',
       'poi_per_capita_h3_l5', 'poi_per_capita_relative_h3_l7_to_h3_l5',
       'region_h3', 'is_null_population', 'cnt_poi_h3_l7_max',
       'cnt_poi_h3_l7_normalized', 'poi_per_capita_h3_l7_max',
       'poi_per_capita_h3_l7_normalized',
       'poi_per_capita_relative_h3_l7_to_h3_l5_max',
       'poi_per_capita_relative_h3_l7_to_h3_l5_normalized', 'h3_polygons'],
      dtype='object')

In [303]:
from shapely import geometry
import plotly.graph_objects as go
center_lon, center_lat = (wkt.loads(df_regions_to_h3[df_regions_to_h3['location']=='Los Angeles']['region_centroid'].iloc[0]).x,
                          wkt.loads(df_regions_to_h3[df_regions_to_h3['location']=='Los Angeles']['region_centroid'].iloc[0]).y)

color_polygons = 'rgba(25,210,242,0.1)'
color_hexagons = 'rgba(25,210,242,0.0)'

import plotly.graph_objects as go

#fig = go.Figure(
fig1 = (
    px.choropleth_mapbox(
        df_regions_to_h3,
        geojson=geojson_object_hexagons,
        locations='h3_l7',
        #color='is_null_population',
        color='poi_per_capita_relative_h3_l7_to_h3_l5_normalized', #'pct_area_residential',
        #color_discrete_map = {'Los Angeles':color_hexagons,'Barcelona':color_hexagons,'London':color_hexagons,'Birmingham':color_hexagons,'Minsk':color_hexagons,'Soligorsk':color_hexagons},
        #color_discrete_map = {0:'#000000',1:'#ff0000'},
        color_continuous_scale=[[0,'#ffffff'],[1,'#ffa23a']], #ffbf65 #ffa23a orange
        #color_continuous_scale='Viridis',
        range_color=(0,max(df_regions_to_h3['poi_per_capita_relative_h3_l7_to_h3_l5_normalized'])),
        center={"lat": center_lat, "lon": center_lon},
        zoom=9,
        #width=1200, 
        height=800,
        opacity=1,
        hover_data={'h3_l7':True,'h3_l5':True,'location':True,'population':True,'category_Randall':True,'cnt_poi_h3_l7':True,'cnt_poi_h3_l7_in_h3_l5':True,'poi_per_capita_h3_l7':True,'poi_per_capita_h3_l5':True,'poi_per_capita_relative_h3_l7_to_h3_l5':True},
        #labels={'h3_l7':'h3_l7'}
    )
)
#fig1.add_trace(
#fig = go.Figure(
fig = (
    px.choropleth_mapbox(
        df_region_polygons,
        geojson=geojson_object_polygons,
        locations='location',
        color='location',
        #color_discrete_map = {'Los Angeles':'#19D2F2','Barcelona':'#19D2F2','London':'#19D2F2','Birmingham':'#19D2F2','Minsk':'#19D2F2','Soligorsk':'#19D2F2'},
        color_discrete_map = {'Los Angeles':color_polygons,'San Francisco':color_polygons,'Barcelona':color_polygons,'London':color_polygons,'Birmingham':color_polygons,'Minsk':color_polygons,'Soligorsk':color_polygons},
        #color_continuous_scale=['#ffffff','#ffa23a'],
        #color_continuous_scale='Viridis',
        #range_color=(0,4),
        center={"lat": center_lat, "lon": center_lon},
        #mapbox_style='carto-positron',
        zoom=9,
        height=800,
        opacity=1,
        hover_data={'location':True},
        #labels={'location'}
    )
)


lats = []
lons = []
names = []

for feature, name in zip(df_region_polygons['exterior_ring_shapely'], df_region_polygons['location']):
    #print('feature: ',feature)
    #print('name: ',name)
    if isinstance(feature, geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        #print('linestring: ', linestring)
        x, y = linestring.xy
        #print('x,y: ',x,y)
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        names = np.append(names, [name]*len(y))
        lats = np.append(lats, None)
        lons = np.append(lons, None)
        names = np.append(names, None)
#print((lats,lons,names))

#fig.add_trace(
fig3 = (
    px.line_geo(#data_frame = pd.DataFrame({'lat':lats,'lon':lons,'name':names}),
                #geojson='geojson_object_polygons',
                lon = lons, 
                lat = lats,
                #color = 'name',
        locations=names,
                markers = True
                #color_discrete_map = {'Los Angeles':color_polygons,'San Francisco':color_polygons,'Barcelona':color_polygons,'London':color_polygons,'Birmingham':color_polygons,'Minsk':color_polygons,'Soligorsk':color_polygons},
                #lat=lats, lon=lons, hover_name=names
                  )
)
#fig.add_trace(
# fig3 = (
#     go.Scattergeo(
#     lon = lons,
#     lat = lats,
#     #geojson=geojson_object_regions_exterior_lines,
#     #locations = 'locations',
#     mode = 'lines',
#     line = dict(width = 4,color = color_polygons),
#     opacity=1
#     #opacity = float(df_flight_paths['cnt'][i]) / float(df_flight_paths['cnt'].max()),
#         )
# )
# fig.add_scattermapbox(
#     #data_frame=df_region_polygons,
#     #colors='location',
#     #color_discrete_map = {'Los Angeles':color_polygons,'San Francisco':color_polygons,'Barcelona':color_polygons,'London':color_polygons,'Birmingham':color_polygons,'Minsk':color_polygons,'Soligorsk':color_polygons},
#     lat = lats,
#     lon = lons,
#     mode = 'lines',
#     #text = texts,
#     marker_size=12,
#     marker_color='rgba(25,210,242)',
#     fillcolor='rgba(25,210,242)'
# )
# fig.add_traces(go.Scattergeo(
#     geojson = geojson_object_regions_exterior_lines,
#     #locationmode = 'USA-states',
#     locations='location',
#     lon = lons,
#     lat = lats,
#     #hoverinfo = 'text',
#     #text = df_airports['airport'],
#     mode = 'lines',
#     line = dict(
#         width = 4,
#         color = 'rgba(25,210,242)'
        
#     )))
# map_stores = px.scatter_mapbox(
#             data_frame = pd.DataFrame({'lat':lats,'lon':lons,'name':names}),
#             lon = 'lon', 
#             lat = 'lat',
#             color = 'name',
#             #hover_name = 'Name',
#             size_max = 20,
#         )

#fig.add_traces(list(fig3.select_traces()))



############
#fig.layout.coloraxis2 = fig1.layout.coloraxis

fig3.update_traces(line=dict(color='rgba(25,210,242,1)', width=4))
fig3.update_traces(marker_line_width=4,marker_line_color='rgba(25,210,242,1)')

fig.add_traces(list(fig3.select_traces()))
#fig.update_traces(line=dict(color='rgba(25,210,242,1)', width=4))


# fig1.update_layout(showlegend=False)

# fig.layout.coloraxis2 = fig1.layout.coloraxis
# fig.add_trace(fig1.data[0])
# fig.data[8]['coloraxis'] = 'coloraxis2'
# fig.data[8]['z'] = df_regions_to_h3['poi_per_capita_relative_h3_l7_to_h3_l5_normalized']
# #fig.layout.coloraxis2.colorbar.x = -0.2
# fig.layout.coloraxis2.colorbar.title.text = 'PCT h3l7/h3l5'

#fig.add_trace(list(fig3.select_traces()))


#fig.update_layout(newshape_line_color='rgba(25,210,242,1)',newshape_line_width=4) #['#ffffff','#ffa23a'])
#fig3.update_traces(marker_line_width=4,marker_line_color='rgba(25,210,242,1)')

#fig.add_trace(fig3.data[0])
#fig.update_traces(color_continuous_scale=['#ffffff','#ffa23a'])
#fig.update_layout(colorscale_sequential=[[0,'#ffffff'],[1,'#ffa23a']])


# fig['data'][1]['marker'] = {'color' : np.array(df_region_polygons['location']),
#                             'coloraxis' : 'coloraxis2',
#                             'opacity' : 1,
#                             'sizemode' : 'area',
#                             'sizeref' : .01,
#                             'autocolorscale' : False
#                            }
#fig.layout.coloraxis2.colorbar.x = -0.2
#fig.update_layout(colorscale_sequential=[[0, 'rgb(220,220,220)'], [0.2, 'rgb(245,195,157)'], [0.4, 'rgb(245,160,105)'], [1, 'rgb(178,10,28)'], ]) #['#ffffff','#ffa23a'])

#fig.update_geos(fitbounds="locations")
fig.update_layout(mapbox_style="carto-positron")
# fig.update_layout(legend=dict(
#     title="Locations",
#     font=dict(
#         family="Consolas",
#         size=10,
#         color="black"
#     )
# ))



# 'open-street-map', 'white-bg', 'carto-positron', 'carto-darkmatter', 'stamen- terrain', 'stamen-toner', 'stamen-watercolor'
fig3.show()


In [302]:
fig3.data


(Scattergeo({
     'geo': 'geo',
     'hovertemplate': 'lat=%{lat}<br>lon=%{lon}<br>locations=%{location}<extra></extra>',
     'lat': array([51.7298297, 51.3866037, 51.1640971, 51.201113, 51.5431696, 51.8105622,
                   51.7298297, None, 52.6693307, 52.4104047, 52.2812087, 52.3584361,
                   52.6526719, 52.6943071, 52.6693307, None, 52.8686898, 52.7774063,
                   52.6292865, 52.6959194, 52.8703477, 52.8686898, None, 54.0744289,
                   53.8951654, 53.7460014, 53.7995669, 54.0276688, 54.0744289, None,
                   37.7601185, 37.691688, 37.5502822, 37.4879193, 37.4277254, 37.4696696,
                   37.5404826, 37.6536437, 37.7829147, 37.9422909, 38.0667325, 38.0786246,
                   38.0483501, 37.9606999, 37.8404166, 37.7601185, None, 34.0412527,
                   34.0025533, 33.8715278, 33.7486871, 33.6866649, 33.7330527, 33.8681071,
                   34.0230434, 34.2367495, 34.3660829, 34.3151722, 34.0412527, None,
     

In [ ]:
geojson_object_hexagons
